In [140]:
import numpy as np
import copy

In [141]:
class LinearExpression:
    @staticmethod
    def get_variable(idx):
        if idx < 0:
            raise ValueError(f'Negative index: {idx}')
        coeffs = np.zeros(idx + 1)
        coeffs[idx] = 1
        return LinearExpression(coeffs)
    
    @staticmethod
    def get_variables(n):
        return [LinearExpression.get_variable(i) for i in range(n)]
    
    @staticmethod
    def get_constant(c):
        return LinearExpression([], c)   
    
    
    
    def extend_to_length(self, other_n):
        if other_n <= self.n:
            return
        
        to_add = other_n - self.n
        self.coeffs = np.array(list(self.coeffs) + list(np.zeros(to_add)), dtype='float32')
        self.n = other_n
        
    
    def __init__(self, 
                 coeffs,
                 constant = 0):
        self.n = len(coeffs)
        self.constant = constant
        self.coeffs = np.array(coeffs, dtype='float32')
    
    def __str__(self):
        variables = ' + '.join(f'{c}*x_{i}' if c != 1 else f'x_{i}' for i, c in filter(lambda x: x[1] != 0, enumerate(self.coeffs)))
        constant = str(self.constant)
        if self.n == 0 and self.constant == 0:
            return '0'
        if self.n == 0:
            return constant
        if self.constant == 0:
            return variables
        return variables + ' + ' + constant
    
    def __repr__(self):
        return self.__str__()
    
    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        if idx > self.n - 1:
            return LinearExpression.get_constant(0)
        new_coeffs = np.zeros(idx + 1)
        new_coeffs[idx] = self.coeffs[idx]
        new = LinearExpression(new_coeffs)
        return new
    
    def contains(self, other):
        if isinstance(other, LinearExpression):
            if len(self) < len(other):
                return false
            self_coeffs = self.coeffs
            other_c = copy.deepcopy(other)
            other_c.extend_to_length(len(self))
            other_coeffs = other_c.coeffs
            
            dot_val = np.count_nonzero(np.dot(self_coeffs, other_coeffs) != 0)
            other_val = np.count_nonzero(other_coeffs != 0)
            
            return dot_val == other_val
        else:
            # treat as integer
            return self.coeffs[other] != 0 if other <= len(self) - 1 else False
    
    def coeff_at_variable(self, var):
        
        if isinstance(var, LinearExpression):
            
            # Assumes that this is a variable
            var_idx = np.argwhere(var.coeffs != 0)[0][0]
            
            return self.coeffs[var_idx]
        else:
            
            return self.coeffs[var] if var <= len(self) - 1 else 0
        
    
    def bin_op(self, other, op):
        if not isinstance(other, LinearExpression):
            self.constant = op(self.constant, other)
            return        
        self.constant = op(self.constant, other.constant)
        if self.n == other.n:
            self.coeffs = op(self.coeffs, other.coeffs)
        elif self.n < other.n:
            self.coeffs = list(op(self.coeffs, other.coeffs[:self.n])) + list(op(np.zeros(other.n), other.coeffs))[self.n:]            
        else:
            self.coeffs = list(op(self.coeffs[:other.n], other.coeffs)) + list(op(self.coeffs, np.zeros(self.n)))[other.n:]
        self.coeffs = np.array(self.coeffs, dtype='float32')
        self.n = len(self.coeffs)
            
    
    def __iadd__(self, other):
        self.bin_op(other, lambda a, b: a + b)
        return self
            
    def __add__(self, other):
        new = copy.deepcopy(self)
        new += other
        return new
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __isub__(self, other):
        self.bin_op(other, lambda a, b: a - b)
        return self
    
    def __sub__(self, other):
        new = copy.deepcopy(self)
        new -= other
        return new
          
    def __rsub__(self, other):
        return (-self).__add__(other)
        
    def __imul__(self, other):
#         print(f'Multiplying {self} with {other}')
        if isinstance(other, LinearExpression):
            if (other.coeffs != 0).any():
                raise ValueError(f'Multiplication with non-constants is not yet supported.')
            c = other.constant
        else:
            c = other
        self.coeffs *= c
        self.constant *= c
        return self
        
    def __mul__(self, other):
#         print(f'Multiplying {self} with {other}')
        new = copy.deepcopy(self)
        new *= other
        return new
    
    def __rmul__(self, other):
#         print(f'Multiplying {self} with {other}')
        return self.__mul__(other)
    
    def __itruediv__(self, other):
#         print('idiv')
        if isinstance(other, LinearExpression):
            if (other.coeffs != 0).any():
                raise ValueError(f'Division with non-constants is not yet supported.')
            c = other.constant
        else:
            c = other
        self.coeffs /= c
        self.constant /= c
        return self
    
    def __truediv__(self, other):
#         print('div')
        new = copy.deepcopy(self)
        new /= other
        return new
    
    def __neg__(self):
#         print('neg')
        new = copy.deepcopy(self)
        new *= -1
        return new
    
    def __eq__(self, other):
        if not isinstance(other, LinearExpression):
            return False
        if self.constant != other.constant:
#             print(f'Different constants, returning false.')
            return False
               
        shorter = self if len(self) <= len(other) else other
        longer  = self if len(self) >  len(other) else other
        
#         print(f'Comparing shorter: {shorter} with longer {longer}')
        
        shorter_len = len(shorter)
        longer_len  = len(longer)
        
        if (longer.coeffs[shorter_len:] == 1).any():
#             print(f'Rest is not all zeor so returning False')
            return False
        
#         print(f'Finally comparing {shorter.coeffs} with {longer.coeffs[:shorter_len]}')
        return (shorter.coeffs == longer.coeffs[:shorter_len]).all()
        
  
    def __lt__(self, other):
        return LinearInequality(self, other, '<')
    
    def __gt__(self, other):
        return LinearInequality(self, other, '>')
    
    def __le__(self, other):
        return LinearInequality(self, other, '<=')
    
    def __ge__(self, other):
        return LinearInequality(self, other, '>=')


In [142]:
def get_op(op_string):
    if op_string == '>':
        return lambda a, b: a > b
    elif op_string == '<':
        return lambda a, b: a < b
    elif op_string == '>=':
        return lambda a, b: a >= b
    elif op_string == '<=':
        return lambda a, b: a <= b
    raise ValueError(f'Unknown op: {op_string}')
    
def flip_op(op_string):
    if op_string == '>':
        return '<'
    elif op_string == '<':
        return '>'
    elif op_string == '>=':
        return '<='
    elif op_string == '<=':
        return '>='
    raise ValueError(f'Unknown op: {op_string}')
    
def apply_op(op_string, a, b):
    return get_op(op_string)(a, b)

In [143]:
class LinearInequality:
    def __init__(self, left, right, op):
        self.left = copy.deepcopy(left)
        self.right = copy.deepcopy(right)
        self.op_string = op
        self.op_function = get_op(op)
        
    def __str__(self):
        return f'{self.left.__str__()} {self.op_string} {self.right.__str__()}'
        
    def __repr__(self):
        return self.__str__()
    
    def flip_op(self):
        self.op_string   = flip_op(self.op_string)
        self.op_function = get_op(self.op_string)
        
    def normalize(self):
        self.left -= self.right
        self.right = 0
        
    def get_normalized(self):
        new = copy.deepcopy(self)
        new.normalize()
        return new
    
    def flip(self):
        tmp = self.left
        self.left = self.right
        self.right = tmp
        self.flip_op()
    
    def get_flipped(self):
        new = copy.deepcopy(self)
        new.flip()
        return new
       
    
    def __iadd__(self, other):
        self.left += other
        self.right += other
        return self
    
    def __add__(self, other):
        new = copy.deepcopy(self)
        new += other
        return new
    
    def __radd__(self, other):
        self.__add__(other)
    
    def __isub__(self, other):
        self += (-other)
        return self
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return -self + other
    
    def __imul__(self, other):
        self.left *= other
        self.right *= other
        if other < 0:
            self.flip_op()
        return self
    
    def __mul__(self, other):
        new = copy.deepcopy(self)
        new *= other
        return new
    
    def __rmul__(self, other):
        return self*other
    
    def __itruediv__(self, other):
        self.left /= other
        self.right /= other
        if other < 0:
            self.flip_op()
        return self
    
    def __truediv__(self, other):
        new = copy.deepcopy(self)
        new /= other
        return new
    
    def __neg__(self):
        self.left *= -1
        self.right *= -1
        self.flip_op()
        return self
    
    def __eq__(self, other):
        if not isinstance(other, LinearInequality):
            return False
        
        # TODO: Refactor
        
        self_norm  = self.get_normalized()
        other_norm = other.get_normalized()
            
#         print(f'[normalized] Comparing {self_norm} with {other_norm}')
        
        if other_norm.op_string != self_norm.op_string and other_norm.op_string != flip_op(self_norm.op_string):
#             print(f'[normalized] Incompatible signs, returning false')
            return False
        
        if other_norm.op_string != self_norm.op_string:
#             print(f'[normalized] Correcting other sign, now using {other_norm}')
            other_norm *= -1
            
#         print(f'[normalized] Finally comparing {self_norm} with {other_norm}')
#         print(f'[normalized] Namely comparing {self_norm.left} with {other_norm.left}')
        return self_norm.left == other_norm.left


In [144]:
# Immutable because laziness
class InequalitySystem:
    def __init__(self, ineqs=[]):
        # maybe numpy array
        self.ineqs = ineqs
        self.remove_duplicates()
        for ineq in self.ineqs:
            ineq.normalize()
            
        # TODO: Refactor?
        self.variables = []
        seen = set()
        for ineq in self.ineqs:
            for i, c in enumerate(ineq.left.coeffs):
                if c != 0 and i not in seen:
                    seen.add(i)
                    self.variables.append(LinearExpression.get_variable(i))           
    
    def __str__(self):
        eq_part = '\r\n'.join(str(ineq) for ineq in self.ineqs)
        var_part = ', '.join(str(v) for v in self.variables)
        return f'System with inequalities:\r\n{eq_part}\r\ncontaining variables:\r\n{var_part}'
    
    def __repr__(self):
        return self.__str__()
    
    def n(self):
        return len(ineqs)
        
    def remove_duplicates(self):
        # TODO: Use hasing and np.unique or something
        new_ineqs = []
        for ineq in self.ineqs:
            if ineq not in new_ineqs:
                new_ineqs.append(ineq)       
        self.ineqs = new_ineqs
        
    

        
        
    

In [145]:
def remove_variable(system, variable, variable_idx):
#     print(f'Removing variable {variable} from\r\n{system}')
#     print('======================================')
    
    # (1) - Split
    
    # List of ineqs with this variable (non zero coefficient)
    ineqs_with = []
    # List of ineqs without this variable (zero coefficient) - these go on to the next system
    ineqs_without = []
    for ineq in system.ineqs:
        var_coeff = ineq.left.coeff_at_variable(variable)
        if var_coeff != 0:
            ineq -= variable*var_coeff
            ineq.flip()
            ineq /= (-var_coeff)
            ineqs_with.append(ineq)
        else:
            ineqs_without.append(ineq)
    
    print('Ineqs that contain this variable:')
    print('\n'.join(str(i) for i in ineqs_with))
    print('Ineqs that do NOT contain this variable:')
    print('\n'.join(str(i) for i in ineqs_without))
    
    
    # Now find new equations
    new_ineqs = []
    for i in range(len(ineqs_with)):
        ineq1 = ineqs_with[i]
        for j in range(i+1, len(ineqs_with)):
            ineq2 = ineqs_with[j]
            if ineq1.op_string != flip_op(ineq2.op_string):
                break
            
            left =  ineq1.right if ineq1.op_string == '>=' else ineq2.right
            right = ineq1.right if ineq1.op_string == '<=' else ineq2.right
            
            print(f'Based on {ineq1} and {ineq2} we get a new ineq: {left <= right}')
            
            new_ineqs.append(left <= right)
    
    new_ineqs = new_ineqs + ineqs_without
    return InequalitySystem(new_ineqs)
            
            
    
    

# Returns another system, but solved
def fourier_motzkin(system):
    current_system = copy.deepcopy(system) # MEMORY GO BRRRRRRRRRRRR
    current_system = remove_variable(current_system, system.variables[0], 0)

        
    
    # (4) Return system
    return system
        

In [146]:
x0, x1, x2, x3 = LinearExpression.get_variables(4)

In [147]:
system_eqs = [2*x0 + x1 + x3 + x3 >= 0,
              x2 <= 5,
              -5*x0 + 2*x1 >= 3,
              x0 + x1 + 2*x3>= 0]

In [148]:
system = InequalitySystem(system_eqs)

In [149]:
system

System with inequalities:
2.0*x_0 + x_1 + 2.0*x_3 >= 0
x_2 + -5 <= 0
-5.0*x_0 + 2.0*x_1 + -3 >= 0
x_0 + x_1 + 2.0*x_3 >= 0
containing variables:
x_0, x_1, x_3, x_2

In [150]:
fourier_motzkin(system)

Ineqs that contain this variable:
x_0 >= -0.5*x_1 + -1.0*x_3
x_0 <= 0.4000000059604645*x_1 + -0.6
x_0 >= -1.0*x_1 + -2.0*x_3
Ineqs that do NOT contain this variable:
x_2 + -5 <= 0
Based on x_0 >= -0.5*x_1 + -1.0*x_3 and x_0 <= 0.4000000059604645*x_1 + -0.6 we get a new ineq: -0.5*x_1 + -1.0*x_3 <= 0.4000000059604645*x_1 + -0.6
Based on x_0 <= 0.4000000059604645*x_1 + -0.6 and x_0 >= -1.0*x_1 + -2.0*x_3 we get a new ineq: -1.0*x_1 + -2.0*x_3 <= 0.4000000059604645*x_1 + -0.6


System with inequalities:
2.0*x_0 + x_1 + 2.0*x_3 >= 0
x_2 + -5 <= 0
-5.0*x_0 + 2.0*x_1 + -3 >= 0
x_0 + x_1 + 2.0*x_3 >= 0
containing variables:
x_0, x_1, x_3, x_2